Net IDs: hak68, 

# Research Questions


Based on how long a patient stays in hospital and how much they spent, can we predict what factors went into their care?


# Data collection and cleaning

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb


In [22]:
attributes_df = pd.read_csv("../info2950-final-project/Attribute_Description.csv")
medication_df = pd.read_csv("Medication_Data_Kaggle_V1.csv")
socio_eco_df = pd.read_csv("Socio_Economics_Data_Kaggle_V1.csv")
patient_data_training_df = pd.read_csv("Patient_Data_Training_Kaggle_V1.csv")

**Renaming column names to include no spaces for simpler selection**

In [23]:
def rename_column(colname):
  return colname.replace(' ', '_')

medication_df = medication_df.rename(mapper = rename_column, axis = 1)
socio_eco_df = socio_eco_df.rename(mapper = rename_column, axis = 1)
patient_data_training_df = patient_data_training_df.rename(mapper = rename_column, axis = 1)




**Combining into one dataframe**

# Data description

**What are the observations (rows) and the attributes (columns)?**

Each dataframe, medication_df, socio_eco_df, and patient_data_training_df are a part of the same larger set of data but split into different dataframes. This larger dataframe observed certain attributes for each diabetes patient, recognized by their unique patient id (each row is data for one patient). 

The patient_data_training_df has these attributes:
- SID: patient Unique ID
- Year: patient record year
- Age: patient age
- Race: patient race or ethnicity
- Sex: patient's sex
- State: state patient lives in
- Zip Code: patient zip code
- MSA: metropolitan statistical area code
- Enrollment Type Categorized: 
- Enrollment Months
- HCC Score
- Avg. LOS
- Diagnosis
- #Total Claims
- #Hospital OP
- #All Physician OP
- #Inpatient
- #Short Term Stay Hospital
- #Long Term Stay Hospital
- #Rehabilitation Hospital
- #Psychiatric Hospital
- #Readmissions
- #ER
- #ER Admissions
- #SNF
- #Non Swing Bed SNF Claim
- #Swing Bed SNF Claim
- #Home Health
- #Hospice
- #Labs
- #Part B Labs
- #Imaging
- #Part B Imaging
- #Part B E&M
- #Part B Drugs
- #Part B Ambulance
- #Dialysis
- #Part-B Dialysis
- #Rx Claims
- #DME
- #Miscellaneous
- TotalCost_Y_Actual
- TotalCost_Y_Expected

The medication_df has these attributes:

The socio_eco_df has these attributes: 




**Why was this dataset created?**

This data set was created to analyze the factors that go into diabetes care, the demographics of diabetes patients, and how much it cost these patients to get care.

**Who funded the creation of the dataset?**

**What processes might have influenced what data was observed and recorded and what was not?**


**What preprocessing was done, and how did the data come to be in the form that you are using?**


**If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?**


**Where can your raw source data be found, if applicable?**

# Data limitations

# Exploratory data analysis

**Which medications are popular amoung diabetes patients?**
If we know the medication that is most used by diabetes patients, we can compare the attriibuts of patients who take the most popular medication to patients who take the least popular one such as how much they spent total on care, how long they stay in the hospital, how many times readmitted and see if there is any correlation. We can potentially use these correlations as an indicator of how effective a medication is in diabetes care. For example, if patients who take medication A spend twice as long in hospitals, that provides an interesting correlation to examine further. We, however, need to keep in mind that correlation does not equal causation so we might have to reexamine our definition of effectiveness.
As a bonus, we can use information about this medication such as which company makes it and we could research how much that company makes from this medication. We could perhaps find some correlation between the popularity of the diabetes medication and the success of the company that creates it to see which companies make the most off of chronic diabetes. This could lead to a interesting research question about which chronic diseases aremost profitable but we will again need more data from that

In [24]:
medication_df.head()
pop_med_df = np.sum(medication_df[['dapagliflozin', 'exenatide', 'glimepiride', 'glyburide and metformin hydrochloride', 'insulin', 'metformin hydrochloride', 'pioglitazone', 'sitagliptin and metformin hydrochloride']], axis = 0)


plt.bar(['dapagliflozin', 'exenatide', 'glimepiride',\
         'glyburide and metformin hydrochloride', 'insulin',\
         'metformin hydrochloride', 'pioglitazone',\
         'sitagliptin and metformin hydrochloride'],\
        pop_med_df,\
        color = ['red', 'green', 'blue', 'orange', 'purple', 'magenta', 'indigo', 'yellow'])
plt.xticks(rotation = 90)

KeyError: "['glyburide and metformin hydrochloride', 'metformin hydrochloride', 'sitagliptin and metformin hydrochloride'] not in index"

**Impact of quality of area on amount spent on diabetes care:** The Area Deprivation Index (ADI) is a measure of the quality of the area a person lives in in terms of cleanliness, infrastructure, income, education, and employment rates. High ADI means high deprivation in area. It would be helpful to know more about the background of the patients and to see if there is any correlation between the ADI and how much they pay for care. It could answer questions such as do poorer individuals pay more or less for care?

In [ ]:
adi_vs_cost_df = duckdb.sql('SELECT socio_eco_df.ID, \
socio_eco_df.Avg_ADI, patient_data_training_df.TotalCost_Y_Actual AS Total_Cost \
FROM socio_eco_df \
INNER JOIN patient_data_training_df \
ON socio_eco_df.ID = patient_data_training_df.ID ').df()

plt.plot(adi_vs_cost_df['Avg_ADI'],adi_vs_cost_df['Total_Cost'],'b.')
plt.title('ADI vs Cost of Diabetes Care')
plt.xlabel('Area Deprivation Index ( ADI )')
plt.ylabel('Diabetes Care (\$)')

This scatterplot is a bit difficult to analyze as there is not a clear correlation between ADI and money spent. There are some outliers near a deprivation index of 40 but this visualization is not enough to make predictions based off of so we might not use this comparison

**Which age-group gets diabetes the most?**

In [ ]:
#min age = 20
#max age = 102

sns.histplot(patient_data_training_df[['Age']], x = "Age").set(title = "Diabetes by Age")


**How Long on Average Do Diabetes Patients Stay In Hospital?**

In [ ]:
p = duckdb.sql("SELECT AVG(TotalCost_Y_Actual) AS Avg_Cost, Race FROM patient_data_training_df GROUP BY Race").df()

plt.bar(p['Race'], p['Avg_Cost'])
plt.xlabel('Race')
plt.ylabel('Average Total Cost')
plt.title('Patient Data by Race From 2017 - 2021')
plt.xticks(rotation = 45)

*This bar graph was created to show a comparison of how much patients pay in total for their hospital stays and medication on average based on race. The Native American's average total cost significantly outnumbers the other races.*

**Total Cost of Patient Care Per Year**

In [ ]:
plt.bar(patient_data_training_df['Race'], patient_data_training_df['Avg._LOS'])
plt.xlabel('Race')
plt.ylabel('Average Length of Stay')
plt.title('Avg. LOS by Race')
plt.xticks(rotation = 45)

*This bar graph is a comparison of patients' average length of stay in the hospital based on race. This graph exhibits how White people have significantly longer stays in the hospital due to their conditions and 'other races' not defined spend the least amount of time in the hospitals due to their conditions.*

- A research question we are considering is measuring the effectiveness of certain attributes on diabetes care. To do this, we would have to define a measure of effectiveness such as cost or duration of stay, but we think this could lead to us concluding causations from our data. Is this something we should be weary of or how do you reccomend avoiding this?
- Would a multivariate regression where different care attributes are inputs and the cost or duration of stay is an output be appropriate for our research question? 
- Can you reccomend places for us to find good unclean data?